In [ ]:
!pip install transformers

In [ ]:
!wget https://github.com/Infatoshi/fcc-intro-to-llms/blob/main/wizard_of_oz.txt

--2023-10-29 16:07:22--  https://github.com/Infatoshi/fcc-intro-to-llms/blob/main/wizard_of_oz.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277631 (271K) [text/plain]
Saving to: ‘wizard_of_oz.txt.1’

wizard_of_oz.txt.1  100%[===================>] 271.12K  --.-KB/s    in 0.1s    

2023-10-29 16:07:22 (2.67 MB/s) - ‘wizard_of_oz.txt.1’ saved [277631/277631]



In [ ]:
import mmap
import random
import pickle
import argparse

In [ ]:
import math
import torch
import torch.nn as nn
from dataclasses import dataclass

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 64
block_size = 256
max_iters = 3000
learning_rate = 3e-4
eval_iters = 50

In [ ]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

277628
[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', '\ufeff']


In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

#Model

In [ ]:
# config class
@dataclass
class ModelArgs:
    embed_dim : int
    n_layers : int
    n_heads : int
    head_dim : int
    vocab_size : int
    hidden_dim : int
    block_size : int
    norm_eps : int
    max_batch_size : int
    bias : bool = True

In [ ]:
#Attention Modulde
class SelfAttention(nn.Module):

    def __init__(self, args : ModelArgs):

        super().__init__()

        self.n_heads = args.n_heads
        self.head_dim = args.head_dim
        self.embed_dim = args.embed_dim
        self.block_size = args.block_size
        self.args = args

        assert self.embed_dim % self.n_heads == 0
        assert self.n_heads * self.head_dim == self.embed_dim

        self.wq = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.wk = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.wv = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.wo = nn.Linear(self.embed_dim, self.embed_dim, bias=False)

        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')

        if not self.flash:

            print("warning: flash attention is not supported")

            self.register_buffer("bias", torch.tril(torch.ones(self.block_size, self.block_size))
                                 .view(1, 1, self.block_size, self.block_size))

    def forward(self, x : torch.Tensor):

        B, T, C = x.size() # batch size, sequence length, embedding size(embed_dim)

        q = self.wq(x) # queries
        k = self.wk(x) # keys
        v = self.wv(x) # values

        print(q.size(), k.size(), v.size())

        q = q.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2) # (B, T, C) -> (B, nh, T, hs)
        k = k.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2) # (B, T, C) -> (B, nh, T, hs)
        v = v.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2) # (B, T, C) -> (B, nh, T, hs)

        if self.flash:
            print(q.size(), k.size(), v.size())
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True)
        else:
            y = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            y = y.masked_fill(self.bias[:,:,:T, :T] ==0, float('-inf'))
            y = torch.nn.functional.softmax(y, dim=-1)
            y = y @ v
            y = y.transpose(1, 2).contiguous().view(B, T, C)

        output = self.wo(y) #output projection

        return output




# class SelfAttention(nn.Module):

#     def __init__(self, config : ModelArgs):
#         super().__init__()
#         assert config.embed_dim % config.n_heads == 0
#         # key, query, value projections for all heads, but in a batch
#         self.c_attn = nn.Linear(config.embed_dim, 3 * config.embed_dim, bias=config.bias)
#         # output projection
#         self.c_proj = nn.Linear(config.embed_dim, config.embed_dim, bias=config.bias)
#         # regularization
#         self.attn_dropout = nn.Dropout(0.1)
#         self.resid_dropout = nn.Dropout(0.1)
#         self.n_heads = config.n_heads
#         self.embed_dim = config.embed_dim
#         self.dropout = 0.1

#         self.wq = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
#         self.wk = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
#         self.wv = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
#         self.wo = nn.Linear(self.embed_dim, self.embed_dim, bias=False)

#         # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
#         self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
#         if not self.flash:
#             print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
#             # causal mask to ensure that attention is only applied to the left in the input sequence
#             self.register_buffer("bias", torch.tril(torch.ones(block_size, block_size))
#                                         .view(1, 1, block_size, block_size))

#     def forward(self, x):

#         B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

#         # calculate query, key, values for all heads in batch and move head forward to be the batch dim
#         # q, k, v  = self.c_attn(x).split(self.embed_dim, dim=2)

#         q = self.wq(x)
#         k = self.wk(x)
#         v = self.wv(x)

#         k = k.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2) # (B, nh, T, hs)
#         q = q.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2) # (B, nh, T, hs)
#         v = v.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2) # (B, nh, T, hs)

#         print(q.size(), k.size(), v.size())

#         # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
#         if self.flash:
#             print(q.size(), k.size(), v.size())
#             # efficient attention using Flash Attention CUDA kernels
#             y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True)
#         else:
#             # manual implementation of attention
#             att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
#             att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
#             att = nn.functional.softmax(att, dim=-1)
#             att = self.attn_dropout(att)
#             y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
#         y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

#         # output projection
#         y = self.resid_dropout(self.c_proj(y))
#         return y


In [ ]:
class FeedForward(nn.Module):

    def __init__(self, args : ModelArgs):
        super().__init__()

        self.w1 = nn.Linear(args.embed_dim, args.embed_dim * 4, bias = False)
        self.w2 = nn.Linear(args.embed_dim * 4, args.embed_dim, bias = False)
        self.w3 = nn.Linear(args.embed_dim, args.embed_dim * 4, bias = False)

    def forward(self, x) -> torch.Tensor:
        return self.w2(nn.functional.silu(self.w1(x)) * self.w3(x))


In [ ]:
class RMSNorm(torch.nn.Module):

    def __init__(self, embed_dim : int, eps: float = 1e6):
      super().__init__()
      self.eps = eps
      self.weight = nn.Parameter(torch.ones(embed_dim))

    def _norm(self, x):
      return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self , x):
      output = self._norm(x.float()).type_as(x)
      return output * self.weight


# class RMSNorm(torch.nn.Module):
#     def __init__(self, dim: int, eps: float = 1e-6):
#         super().__init__()
#         self.eps = eps
#         self.weight = nn.Parameter(torch.ones(dim))

#     def _norm(self, x):
#         return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

#     def forward(self, x):
#         output = self._norm(x.float()).type_as(x)
#         return output * self.weight

In [ ]:
class TransformerBlock(nn.Module):

    def __init__(self, args: ModelArgs):
      super().__init__()
      self.args = args
      self.n_heads = args.n_heads
      self.embed_dim = args.embed_dim
      self.Attention = SelfAttention(args)
      self.Feed_Forward = FeedForward(args)
      self.Attention_Norm = RMSNorm(self.embed_dim, eps = 1e6)
      self.Feed_Forward_Norm = RMSNorm(self.embed_dim, eps = 1e6)

    def forward(self, x: torch.Tensor):
      x = self.Attention_Norm(x)
      x = x + self.Attention(x)
      x = self.Feed_Forward_Norm(x)
      x = x + self.Feed_Forward(x)
      return x

In [ ]:
class Transformer(nn.Module):

    def __init__(self , args: ModelArgs):
      super().__init__()
      assert args.vocab_size is not None
      self.args = args

      self.transformer = nn.ModuleDict(dict(
          wte = nn.Embedding(args.vocab_size, args.embed_dim),
          wpe = nn.Embedding(args.block_size, args.embed_dim),
          drop = nn.Dropout(0.1),
          h = nn.ModuleList([TransformerBlock(args) for _ in range(args.n_layers)]),
          ln_f = RMSNorm(args.embed_dim, eps =1e6)
      ))

      self.lm_head = nn.Linear(args.embed_dim, args.vocab_size,bias = False )
      self.transformer.wte.weight = self.lm_head.weight
      self.apply(self._init_weights)
      print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))


    def get_num_params(self, non_embedding=True):
          """
          Return the number of parameters in the model.
          For non-embedding count (default), the position embeddings get subtracted.
          The token embeddings would too, except due to the parameter sharing these
          params are actually used as weights in the final layer, so we include them.
          """
          n_params = sum(p.numel() for p in self.parameters())
          if non_embedding:
              n_params -= self.transformer.wpe.weight.numel()
          return n_params


    def _init_weights(self, module):
      if isinstance(module, nn.Linear):
        torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
        if module.bias is not None:
          torch.nn.init.zeros_(module.bias)
      elif isinstance(module, nn.Embedding):
        torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)


    def forward(self, idx, targets = None):
      device = idx.device
      b, t = idx.size()
      assert t <= self.args.block_size, f" cannot forward a sequence with length greater than max_block_size"
      pos = torch.arange(0, t, dtype=torch.long, device=device)

      tok_embed = self.transformer.wte(idx)
      pos_embed = self.transformer.wpe(pos)

      x = self.transformer.drop(tok_embed + pos_embed)
      for block in self.transformer.h:
        x = block(x)
      x = self.transformer.ln_f(x)
      logits = self.lm_head(x)

      if targets is None:
            loss = None
      else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = nn.functional.cross_entropy(logits, targets)

      return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.args.block_size else idx[:, -self.args.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx, targets = None)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = nn.functional.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx



In [ ]:
# # config class
# dataclass
# class ModelArgs:
#     embed_dim : int
#     n_layers : int
#     n_heads : int
#     head_dim : int
#     vocab_size : int
#     hidden_dim : int
#     block_size : int
#     norm_eps : int
#     max_batch_size : int
#     bias : bool = True

model_args = ModelArgs(
    embed_dim = 2048,
    n_layers = 32,
    n_heads = 32,
    head_dim = 64,
    vocab_size = 87,
    hidden_dim = 8192,
    block_size = 256,
    norm_eps = 1e6,
    max_batch_size = 32,
    bias = True
)

In [ ]:
print(model_args)

ModelArgs(embed_dim=2048, n_layers=32, n_heads=32, head_dim=64, vocab_size=87, hidden_dim=8192, block_size=256, norm_eps=1000000.0, max_batch_size=32, bias=True)


# create an instance of the model

In [ ]:
model = Transformer(model_args)

number of parameters: 2147.79M


In [ ]:
model = model.to(device)

In [ ]:
prompts = ['hi']
contexts = [torch.tensor(encode(prompt), dtype=torch.long, device=device) for prompt in prompts]
generated_chars = [decode(model.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist()) for context in contexts]
print(generated_chars)

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())